<a href="https://colab.research.google.com/github/rtajeong/M3_new/blob/main/M3_lab53_naver_movie_rev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

네이버영화평점
==
- 감성분석
- 네이버 영화평점 (Naver sentiment movie corpus v.1.0) 데이터(https://github.com/e9t/nsmc)
- 영화 리뷰 20만건이 저장됨. 각 평가 데이터는 0(부정), 1(긍정)으로 label 됨.

### 한글 자연어 처리
- KoNLPy(“코엔엘파이”라고 읽습니다)는 한국어 정보처리를 위한 파이썬 패키지입니다.
- konlpy 패키지에서 제공하는 Twitter라는 문서 분석 라이브러리 사용 (트위터 분석 뿐 아니라 한글 텍스트
  처리도 가능)
- colab 사용 권장

# 로지스틱회귀를 이용한 감성분석

In [26]:
!pip install konlpy

In [27]:
# Curl:
# curl is a tool to transfer data from or to a server, using one of the supported protocols (HTTP, HTTPS, FTP,
# FTPS, SCP, SFTP, TFTP, DICT, TELNET, LDAP or FILE). The command is designed to work without user interaction.
#
# curl -L : (HTTP/HTTPS) If the server reports that the requested page has moved to a different location
# (indicated with a Location: header and a 3XX response code), this option will make curl redo the request
# on the new place.

In [28]:
# 네이버 영화 평점 데이터 다운로드
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   152  100   152    0     0    496      0 --:--:-- --:--:-- --:--:--   496
100    17    0    17    0     0     13      0 --:--:--  0:00:01 --:--:--    45
100   297  100   297    0     0    163      0  0:00:01  0:00:01 --:--:--     0
100 14.0M  100 14.0M    0     0  4217k      0  0:00:03  0:00:03 --:--:-- 14.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   151  100   151    0     0    482      0 --:--:-- --:--:-- --:--:--   482
100    17    0    17    0     0     13      0 --:--:--  0:00:01 --:--:--     0
100   297  100   297    0     0    160      0  0:00:01  0:00:01 --:--:--   160
100 4827k  100 4827k    0     0  1022k      0  0:00:04  0:00:04 --:--:-- 1986k


In [29]:
import konlpy
import pandas as pd
from konlpy.tag import Twitter, Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline
# import pickle
# import os.path

# 데이터 로드
# keep_default_na = False: use an empty string instead of a NaN when parsing missing values

df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)

df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [30]:
df_train.shape, df_test.shape

((150000, 3), (50000, 3))

In [31]:
text_train, y_train = df_train['document'].values, df_train['label'].values
text_test, y_test = df_test['document'].values, df_test['label'].values

In [32]:
text_train.shape, text_test.shape   # too big

((150000,), (50000,))

In [33]:
text_train, y_train = text_train[:2000], y_train[:2000]
text_test, y_test = text_test[:1000], y_test[:1000]

In [34]:
text_train.shape, text_test.shape

((2000,), (1000,))

In [35]:
from konlpy.tag import Twitter, Okt
def okt_tokenizer(text):
    return Okt().morphs(text)


In [36]:
cv = TfidfVectorizer(tokenizer=okt_tokenizer, max_features = 5000, min_df=5)
x_train = cv.fit_transform(text_train)
x_test = cv.transform(text_test)

lr = LogisticRegression()
result = lr.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [37]:
len(cv.vocabulary_), x_train.shape, x_test.shape

(794, (2000, 794), (1000, 794))

In [38]:
print("훈련 데이터 점수 : ", result.score(x_train, y_train))
print("테스트 데이터 점수 : ", result.score(x_test, y_test))

훈련 데이터 점수 :  0.8555
테스트 데이터 점수 :  0.746


In [39]:
feature_names = cv.get_feature_names_out()
print(feature_names[-10:])

['화' '화면' '화보' '화이팅' '확실히' '후' '후회' '흠' '희망' '히']


# 불용어 처리
- 한국어  불용어 확인은 형태소 분석 라이브러리인 KoLPy 를 이용하면 됨.
- (예) 한국어 품사 중 조사를 추출하는 예
- pos (part-of-speech): 품사 (명사, 동사, ...)

In [40]:
Okt().morphs("텍스트 데이터를 이용한 불용어 사전을 구축")

['텍스트', '데이터', '를', '이용', '한', '불', '용어', '사전', '을', '구축']

In [41]:
Okt().pos("텍스트 데이터를 이용한 불용어 사전을 구축")

[('텍스트', 'Noun'),
 ('데이터', 'Noun'),
 ('를', 'Josa'),
 ('이용', 'Noun'),
 ('한', 'Josa'),
 ('불', 'Noun'),
 ('용어', 'Noun'),
 ('사전', 'Noun'),
 ('을', 'Josa'),
 ('구축', 'Noun')]

In [42]:
Okt().pos("텍스트 데이터를 이용한 불용어 사전을 구축", norm=True)

[('텍스트', 'Noun'),
 ('데이터', 'Noun'),
 ('를', 'Josa'),
 ('이용', 'Noun'),
 ('한', 'Josa'),
 ('불', 'Noun'),
 ('용어', 'Noun'),
 ('사전', 'Noun'),
 ('을', 'Josa'),
 ('구축', 'Noun')]

In [43]:
Okt().nouns("텍스트 데이터를 이용한 불용어 사전을 구축")

['텍스트', '데이터', '이용', '불', '용어', '사전', '구축']

- norm: 오타수정, stem: 어근 찾기

In [48]:
from konlpy.tag import Okt
okt = Okt()
word_tags = okt.pos("텍스트 데이터를 이용한 불용어 사전을 구축, ~ ! == ",
                       norm=True, stem=True)
print(word_tags)
words = [word[0] for word in word_tags if word[1]=="Noun"]
print (words)

[('텍스트', 'Noun'), ('데이터', 'Noun'), ('를', 'Josa'), ('이용', 'Noun'), ('한', 'Josa'), ('불', 'Noun'), ('용어', 'Noun'), ('사전', 'Noun'), ('을', 'Josa'), ('구축', 'Noun'), (',', 'Punctuation'), ('~', 'Punctuation'), ('!', 'Punctuation'), ('==', 'Punctuation')]
['텍스트', '데이터', '이용', '불', '용어', '사전', '구축']


In [57]:
def okt_tokenizer2(text):
    word_tags = okt.pos(text, norm=True, stem=True)
    words = [word[0] for word in word_tags if word[1]!="Josa" and word[1]!="Punctuation"]
    return words

In [58]:
cv = TfidfVectorizer(tokenizer=okt_tokenizer2, max_features = 500, min_df=5)
x_train = cv.fit_transform(text_train)
x_test = cv.transform(text_test)

lr = LogisticRegression()
lr.fit(x_train,y_train)
print("훈련 데이터 점수 : ", lr.score(x_train, y_train))
print("테스트 데이터 점수 : ", lr.score(x_test, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


훈련 데이터 점수 :  0.8315
테스트 데이터 점수 :  0.761


# Exercise